# Opening a New  GYM in Mumbai, India
Build a dataframe of neighborhoods in Mumbai, India by web scraping the data from Wikipedia page.

Get the geographical coordinates of the neighborhoods.

Obtain the venue data for the neighborhoods from Foursquare API.

Explore and cluster the neighborhoods.

Select the best cluster to open a new Gym.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


# Scraping Data from Wikipedia

In [2]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Mumbai").text


In [3]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [4]:
soup

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Category:Suburbs of Mumbai - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XhC@PApAAEYAAGzxj@EAAAAJ","wgCSPNonce":!1,"wgCanonicalNamespace":"Category","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":14,"wgPageName":"Category:Suburbs_of_Mumbai","wgTitle":"Suburbs of Mumbai","wgCurRevisionId":289157432,"wgRevisionId":289157432,"wgArticleId":8732879,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Commons category link is on Wikidata","Neigh

In [5]:
neighborhoodList = []

In [6]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [7]:
mumbai_df = pd.DataFrame({"Neighborhood": neighborhoodList})

mumbai_df.head()


,Neighborhood
0,Andheri
1,Anushakti Nagar
2,Baiganwadi
3,Bandra
4,Bhandup


# Getting Coordinates for every Neighborhood

In [8]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai , India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [9]:
coords = [ get_latlng(neighborhood) for neighborhood in mumbai_df["Neighborhood"].tolist() ]

In [10]:
coords

[[19.118459378296492, 72.84176321065843],
 [19.042830000000038, 72.92734000000007],
 [19.062940000000026, 72.92663000000005],
 [19.054370000000063, 72.84017000000006],
 [19.145560000000046, 72.94856000000004],
 [19.229360000000042, 72.85751000000005],
 [19.208660000000066, 72.82612000000006],
 [19.06218000000007, 72.90241000000003],
 [19.250030000000038, 72.85907000000003],
 [19.224690000000066, 72.86605000000003],
 [19.212750000000028, 73.08324000000005],
 [19.00534722389655, 72.85580272012932],
 [19.08652321008152, 72.90900774216628],
 [19.164550000000077, 72.84946000000008],
 [18.959290000000067, 72.83108000000004],
 [19.137920000000065, 72.84941000000003],
 [19.014920000000075, 72.84522000000004],
 [18.953937419095155, 72.82036732944775],
 [19.21198153260436, 72.83757275783374],
 [19.131380000000036, 72.93568000000005],
 [19.127580000000023, 72.82539000000008],
 [19.064980000000048, 72.88069000000007],
 [19.21198153260436, 72.83757275783374],
 [19.048530000000028, 72.93222000000003

In [11]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [12]:
mumbai_df['Latitude'] = df_coords['Latitude']
mumbai_df['Longitude'] = df_coords['Longitude']

In [13]:
# check the neighborhoods and the coordinates
print(mumbai_df.shape)
mumbai_df

(41, 3)


,Neighborhood,Latitude,Longitude
0,Andheri,19.118459,72.841763
1,Anushakti Nagar,19.042830,72.927340
2,Baiganwadi,19.062940,72.926630
3,Bandra,19.054370,72.840170
4,Bhandup,19.145560,72.948560
5,Borivali,19.229360,72.857510
6,Charkop,19.208660,72.826120
7,Chembur,19.062180,72.902410
8,Dahisar,19.250030,72.859070
9,Devipada,19.224690,72.866050


In [14]:
# save the DataFrame as CSV file
mumbai_df.to_csv("mumbai_df.csv", index=False)


In [15]:
# get the coordinates of Mumbai, India
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai, India 18.9387711, 72.8353355.


In [16]:
# create map of Mumbai using latitude and longitude values
map_mumbai = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(mumbai_df['Latitude'], mumbai_df['Longitude'], mumbai_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_mumbai)  
    
map_mumbai

In [18]:
# save the map as HTML file
map_mumbai.save('map_mumbai.html')

In [19]:
# define Foursquare Credentials and Version
CLIENT_ID = 'Z4A4QJX4ILQY5K0PCCEBGQSC2K4LZJGFH3N24WFTZ4V5GMFR' # your Foursquare ID
CLIENT_SECRET = 'E5XKEDSOCMSRFLDTWDSHRRXIFIKC3WETYCLFKBT0GFBUYGGV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Z4A4QJX4ILQY5K0PCCEBGQSC2K4LZJGFH3N24WFTZ4V5GMFR
CLIENT_SECRET:E5XKEDSOCMSRFLDTWDSHRRXIFIKC3WETYCLFKBT0GFBUYGGV


In [20]:
radius = 2000
LIMIT = 50

venues = []

for lat, long, neighborhood in zip(mumbai_df['Latitude'], mumbai_df['Longitude'], mumbai_df['Neighborhood']):

    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']

    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [21]:
results

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4b7fecb5f964a520c04430e3',
   'name': 'Worli Sea Face',
   'location': {'address': 'Worli',
    'crossStreet': 'Khan Abdul Gaffar Khan Road',
    'lat': 19.009216497075652,
    'lng': 72.8150224685669,
    'labeledLatLngs': [{'label': 'display',
      'lat': 19.009216497075652,
      'lng': 72.8150224685669}],
    'distance': 278,
    'postalCode': '400018',
    'cc': 'IN',
    'city': 'Mumbai',
    'state': 'Mahārāshtra',
    'country': 'India',
    'formattedAddress': ['Worli (Khan Abdul Gaffar Khan Road)',
     'Mumbai 400018',
     'Mahārāshtra',
     'India']},
   'categories': [{'id': '4bf58dd8d48988d165941735',
     'name': 'Scenic Lookout',
     'pluralName': 'Scenic Lookouts',
     'shortName': 'Scenic Lookout',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/sceniclookout_',
      

In [65]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df

(1654, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Andheri,19.118459,72.841763,Merwans Cake shop,19.119300,72.845418,Bakery
1,Andheri,19.118459,72.841763,Radha Krishna Veg Restaurant,19.115130,72.843060,Indian Restaurant
2,Andheri,19.118459,72.841763,Naturals,19.111204,72.837255,Ice Cream Shop
3,Andheri,19.118459,72.841763,Narayan Sandwich,19.121398,72.850270,Sandwich Place
4,Andheri,19.118459,72.841763,Temptations,19.113767,72.841337,Ice Cream Shop
5,Andheri,19.118459,72.841763,Shawarma Factory,19.124591,72.840398,Falafel Restaurant
6,Andheri,19.118459,72.841763,Starbucks Coffee : A Tata Alliance,19.114569,72.836205,Coffee Shop
7,Andheri,19.118459,72.841763,Quench- All Day Pub,19.114538,72.836204,Pub
8,Andheri,19.118459,72.841763,Amar Juice Centre,19.109553,72.837021,Juice Bar
9,Andheri,19.118459,72.841763,Joey's Pizza,19.126762,72.830001,Pizza Place


In [64]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Andheri,50,50,50,50,50,50
Anushakti Nagar,18,18,18,18,18,18
Baiganwadi,9,9,9,9,9,9
Bandra,50,50,50,50,50,50
Bhandup,24,24,24,24,24,24
Borivali,50,50,50,50,50,50
Charkop,50,50,50,50,50,50
Chembur,50,50,50,50,50,50
Dahisar,50,50,50,50,50,50


In [24]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 158 uniques categories.


In [25]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Bakery', 'Indian Restaurant', 'Ice Cream Shop', 'Sandwich Place',
       'Falafel Restaurant', 'Coffee Shop', 'Pub', 'Juice Bar',
       'Pizza Place', 'Multiplex', 'Fast Food Restaurant',
       'Breakfast Spot', 'Seafood Restaurant', 'Café', 'Snack Place',
       'Chinese Restaurant', 'Maharashtrian Restaurant',
       'American Restaurant', 'Cocktail Bar', 'Bar', 'Diner',
       'Gym / Fitness Center', 'BBQ Joint', 'Spa', "Women's Store",
       'Department Store', 'Food', 'Plaza', 'Sports Bar',
       'Mughlai Restaurant', 'Park', 'Platform',
       'Paper / Office Supplies Store', 'Music Venue', 'Hot Dog Joint',
       'Food Truck', 'Gym', 'Lounge', 'Dessert Shop', 'Sports Club',
       'Gourmet Shop', 'Korean Restaurant', 'Deli / Bodega', 'Restaurant',
       'German Restaurant', 'Asian Restaurant', 'Salad Place',
       'French Restaurant', 'Sushi Restaurant',
       'Modern European Restaurant'], dtype=object)

# Analyzing each Neighborhood 

In [27]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(1654, 159)


,Neighborhoods,Afghan Restaurant,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Beach,Bed & Breakfast,Beer Garden,Big Box Store,Bistro,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General College & University,General Entertainment,German Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,Office,Opera House,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pub,Punjabi Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Roof Deck,Salad Place,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Train,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Wine Bar,Women's Store
0,Andheri,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [28]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped


(40, 159)


,Neighborhoods,Afghan Restaurant,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Beach,Bed & Breakfast,Beer Garden,Big Box Store,Bistro,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General College & University,General Entertainment,German Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,Office,Opera House,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pub,Punjabi Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Roof Deck,Salad Place,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Train,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Wine Bar,Women's Store
0,Andheri,0.00,0.020000,0.000000,0.0,0.00000,0.00,0.020000,0.020000,0.020,0.00,0.00,0.00000,0.000000,0.00,0.000000,0.00,0.00,0.02,0.000000,0.00,0.00,0.000000,0.060000,0.00,0.080000,0.000000,0.02,0.020000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00000,0.00,0.000000,0.00000,0.00,0.02000,0.00,0.000000,0.00,0.020000,0.00,0.00,0.020000,0.000000,0.020000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00000,0.000000,0.0,0.00000,0.00,0.000000,0.00,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.020000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.080000,0.160000,0.00,0.00,0.00,0.00,0.000000,0.040000,0.00,0.00,0.00,0.000000,0.02,0.000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.020000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.060000,0.000000,0.00,0.000000,0.060000,0.00,0.000,0.00,0.00,0.000000,0.000000,0.00,0.020000,0.000,0.000,0.06,0.000000,0.000000,0.00,0.060000,0.00,0.00,0.000000,0.020000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.02
1,Anushakti Nagar,0.00,0.000000,0.000000,0.0,0.00000,0.00,0.000000,0.000000,0.000,0.00,0.00,0.00000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.111111,0.00,0.111111,0.000000,0.00,0.055556,0.00,0.000000,0.00,0.00,0.00,0.00,0.00000,0.00,0.000000,0.00000,0.00,0.00000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.055556,0.00,0.00,0.055556,0.00000,0.000000,0.0,0.00000,0.00,0.000000,0.00,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.055556,0.000000,0.00,0.000000,0.000000,0.111111,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00

In [30]:
len(kl_grouped[kl_grouped["Gym"] > 0])

18

In [31]:
mumbai_mall = kl_grouped[["Neighborhoods","Gym"]]

In [32]:
mumbai_mall

,Neighborhoods,Gym
0,Andheri,0.000000
1,Anushakti Nagar,0.000000
2,Baiganwadi,0.111111
3,Bandra,0.000000
4,Bhandup,0.000000
5,Borivali,0.040000
6,Charkop,0.040000
7,Chembur,0.020000
8,Dahisar,0.060000
9,Devipada,0.040000


In [54]:
# set number of clusters
kclusters = 2

mumbai_clustering = mumbai_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mumbai_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 1, 0, 0, 1, 1, 0, 1, 1], dtype=int32)

In [55]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
mumbai_merged = mumbai_mall.copy()

# add clustering labels
mumbai_merged["Cluster Labels"] = kmeans.labels_

In [56]:
mumbai_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
mumbai_merged.head()

,Neighborhood,Gym,Cluster Labels
0,Andheri,0.000000,0
1,Anushakti Nagar,0.000000,0
2,Baiganwadi,0.111111,1
3,Bandra,0.000000,0
4,Bhandup,0.000000,0


In [57]:
# merge mumbai_grouped with mumbai_data to add latitude/longitude for each neighborhood
mumbai_merged = mumbai_merged.join(mumbai_df.set_index("Neighborhood"), on="Neighborhood")

print(mumbai_merged.shape)
mumbai_merged.head() # check the last columns!

(40, 5)


,Neighborhood,Gym,Cluster Labels,Latitude,Longitude
0,Andheri,0.000000,0,19.118459,72.841763
1,Anushakti Nagar,0.000000,0,19.042830,72.927340
2,Baiganwadi,0.111111,1,19.062940,72.926630
3,Bandra,0.000000,0,19.054370,72.840170
4,Bhandup,0.000000,0,19.145560,72.948560


In [58]:
# sort the results by Cluster Labels
print(mumbai_merged.shape)
mumbai_merged.sort_values(["Cluster Labels"], inplace=True)
mumbai_merged

(40, 5)


,Neighborhood,Gym,Cluster Labels,Latitude,Longitude
0,Andheri,0.000000,0,19.118459,72.841763
21,Kurla,0.000000,0,19.064980,72.880690
22,Mahavir Nagar (Kandivali),0.000000,0,19.211982,72.837573
23,Mankhurd,0.000000,0,19.048530,72.932220
24,"Matharpacady, Mumbai",0.000000,0,19.044920,72.867205
25,Mira Road,0.025000,0,19.280032,72.867932
27,Mumbra,0.000000,0,19.190540,73.022660
28,Pestom sagar,0.020000,0,19.070640,72.902170
29,Seven Bungalows,0.000000,0,19.131342,72.816342
30,Shil Phata,0.000000,0,19.146580,73.040050


In [59]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mumbai_merged['Latitude'], mumbai_merged['Longitude'], mumbai_merged['Neighborhood'], mumbai_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


In [60]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

In [61]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 0]

,Neighborhood,Gym,Cluster Labels,Latitude,Longitude
0,Andheri,0.000000,0,19.118459,72.841763
21,Kurla,0.000000,0,19.064980,72.880690
22,Mahavir Nagar (Kandivali),0.000000,0,19.211982,72.837573
23,Mankhurd,0.000000,0,19.048530,72.932220
24,"Matharpacady, Mumbai",0.000000,0,19.044920,72.867205
25,Mira Road,0.025000,0,19.280032,72.867932
27,Mumbra,0.000000,0,19.190540,73.022660
28,Pestom sagar,0.020000,0,19.070640,72.902170
29,Seven Bungalows,0.000000,0,19.131342,72.816342
30,Shil Phata,0.000000,0,19.146580,73.040050


In [62]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 1]

,Neighborhood,Gym,Cluster Labels,Latitude,Longitude
9,Devipada,0.040000,1,19.22469,72.86605
6,Charkop,0.040000,1,19.20866,72.82612
5,Borivali,0.040000,1,19.22936,72.85751
33,Tilak Nagar (Mumbai),0.050000,1,18.99616,72.85279
2,Baiganwadi,0.111111,1,19.06294,72.92663
26,Mulund,0.040000,1,19.17183,72.95565
8,Dahisar,0.060000,1,19.25003,72.85907


#Observations:

Most of the Gym are concentrated in the central area of Mumbai city, with the highest number in cluster 1 and moderate number in cluster 0. This represents a great opportunity and high potential areas to open new gym  as there is very little to no competition from gyms . Meanwhile, Gym  in cluster 1 are likely suffering from intense competition due to oversupply and high concentration of gyms .

From another perspective, this also shows that the oversupply of gyms  mostly happened in the central area of the city, with the suburb area still have very few gyms. Therefore, this project recommends property developers to capitalize on these findings to open new gyms  in neighborhoods in cluster 0 with little to no competition.

